In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import os
import random

In [2]:
DEFAULT_RANDOM_SEED = 42

def set_all_seeds(seed=DEFAULT_RANDOM_SEED):

    # python's seeds
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

set_all_seeds(seed=DEFAULT_RANDOM_SEED)

**Пояснение к решению**

На самом деле ничего интересного тут нет
1) Простая предобработка данных:
   
   * заполняем NaN
   * переводим возраст в месяцы
   * при помощи label encoder переводим категориальные признаки в числовые
   * Разбиваем дату на год, месяц, день, часы и минуты.
   * Вау тут даже есть генерация новой фичи!!! Если выделить выходные дни отдельно, результат чуть улучшится
     
2) Модель
   * За основу взят случайный лес, так как для сырых данных хочется использовать сильную модель, с высокой дисперсией, чтоб регуляризацию дополнительно не докидывать
   * Используем стратифицированную к-валидацию
   * Параметры для модели были подогнаны при помощи грид серча, так что выжать что-то дополнительное не получится

In [3]:
def new_preprocess_data(data):


    # Обработка пропусков
    data.fillna({'SexuponOutcome': 'Unknown', 'AgeuponOutcome': '0 years'}, inplace=True)
    data.drop(['Name', 'ID'], axis= 1, inplace= True)
    # Преобразование возраста в месяцы
    def convert_age_to_months(age_str):
        number, unit = age_str.split()
        number = int(number)
        if 'year' in unit:
            return number * 12
        elif 'month' in unit:
            return number
        elif 'week' in unit:
            return number / 4
        elif 'day' in unit:
            return number / 30
        return 0

    data['AgeuponOutcome'] = data['AgeuponOutcome'].apply(convert_age_to_months)

    # Извлечение признаков из даты
    data['DateTime'] = pd.to_datetime(data['DateTime'])
    data['Year'] = data['DateTime'].dt.year
    data['Month'] = data['DateTime'].dt.month
    data['DayOfWeek'] = data['DateTime'].dt.dayofweek
    data['Hour'] = data['DateTime'].dt.hour
    data['Minute'] = data['DateTime'].dt.minute
    data.drop(columns=['DateTime'], inplace= True)

    data['Is_Weekend'] = 1/data['DayOfWeek'].apply(lambda x: 2 if x >= 5 else 1)

    # Преобразование категориальных признаков
    label_encoders = {}
    categorical_features = ['AnimalType', 'SexuponOutcome', 'Breed', 'Color']
    for feature in categorical_features:
        le = LabelEncoder()
        data[feature] = le.fit_transform(data[feature])

    X = data.drop(columns=['Outcome'], errors= 'ignore')
    y = data['Outcome'] if 'Outcome' in data.columns else None
    data['SexuponOutcome'] = 1/(data['SexuponOutcome'] + 1)
    return X, y

def new_train_random_forest(X, y):
    
    # Стратифицированная k-fold кросс-валидация
    skf = StratifiedKFold(n_splits= 20, shuffle= True, random_state= 42)
    model = RandomForestClassifier(n_estimators= 200, random_state= 42)

    accuracies = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Обучение модели
        model.fit(X_train, y_train)

        # Предсказание и оценка
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    print(f'Average Accuracy: {np.mean(accuracies):.2f}')
    return model

def new_predict_test_file(model, df_test, output_file):
    ids = df_test['ID']
    
    # Предобработка тестового файла
    X_test, _ = new_preprocess_data(df_test)

    # Получение предсказаний
    predictions = model.predict(X_test)
    
    output_df = pd.DataFrame({
        'ID': ids,
        'Outcome': predictions
    })
    output_df.to_csv(output_file, index=False)

In [4]:
df_train = pd.read_csv('/kaggle/input/animal-shelter-log/train.csv')
df_test = pd.read_csv('/kaggle/input/animal-shelter-log/test.csv')

X, y = new_preprocess_data(df_train)
model = new_train_random_forest(X, y)

new_predict_test_file(model, df_test, 'submission.csv')

Average Accuracy: 0.68
